check how many vocab words appear in our irf model

6/28/2021


In [1]:
import pandas as pd
import os,glob

In [17]:
# glob.glob('../data/processed/fig4_modelling/irf_manual_scc/*can*')

In [21]:
tissue_to_res = {
    'GDSD6':['../data/processed/fig4_modelling/irf_manual/test_GDSD6_GDSD6_boosted_stability_score.csv',
 '../data/processed/fig4_modelling/irf_manual/test_GDSD6_GDSD6_stability_score.csv',
 '../data/processed/fig4_modelling/irf_manual/test_GDSD6_purple_boosted_stability_score.csv',
 '../data/processed/fig4_modelling/irf_manual/test_GDSD6_purple_stability_score.csv'],
    'Colon':['../data/processed/fig4_modelling/irf_manual/test_Colon_green_stability_score.csv',
 '../data/processed/fig4_modelling/irf_manual/test_Colon_green_boosted_stability_score.csv'],
    'Melanocytes':['../data/processed/fig4_modelling/irf_manual/test_Melanocytes_blue_stability_score.csv',
 '../data/processed/fig4_modelling/irf_manual/test_Melanocytes_Melanocytes_boosted_stability_score.csv',
 '../data/processed/fig4_modelling/irf_manual/test_Melanocytes_Melanocytes_stability_score.csv',
 '../data/processed/fig4_modelling/irf_manual/test_Melanocytes_blue_boosted_stability_score.csv'],
    'GM12878':['../data/processed/fig4_modelling/irf_manual/test_GM12878_grey_stability_score.csv',
 '../data/processed/fig4_modelling/irf_manual/test_GM12878_grey_boosted_stability_score.csv',
 '../data/processed/fig4_modelling/irf_manual/test_GM12878_GM12878_stability_score.csv',
 '../data/processed/fig4_modelling/irf_manual/test_GM12878_GM12878_boosted_stability_score.csv'],
    'MM':['../data/processed/fig4_modelling/irf_manual_mm/diff_cancer_stability_score.csv',
 '../data/processed/fig4_modelling/irf_manual_mm/expr1_cancer_boosted_stability_score.csv',
 '../data/processed/fig4_modelling/irf_manual_mm/diff_cancer_boosted_stability_score.csv',
 '../data/processed/fig4_modelling/irf_manual_mm/expr_cancer_stability_score.csv',
 '../data/processed/fig4_modelling/irf_manual_mm/expr_cancer_boosted_stability_score.csv',
 '../data/processed/fig4_modelling/irf_manual_mm/expr1_cancer_stability_score.csv'],
    'SCC':['../data/processed/fig4_modelling/irf_manual_scc/diff_cancer_stability_score.csv',
 '../data/processed/fig4_modelling/irf_manual_scc/diff_cancer_boosted_stability_score.csv',
 '../data/processed/fig4_modelling/irf_manual_scc/expr_cancer_stability_score.csv',
 '../data/processed/fig4_modelling/irf_manual_scc/expr_cancer_boosted_stability_score.csv']
    
}




In [22]:
vocab_df = pd.read_csv('../data/processed/mpra/results_df_piv_match_count.csv',index_col=0)
vocab_df


,vocab,tissue,mpra_tissue,class,instance,count,consensus_class,tf1,tf2
0,ARID5B::KLF5,GDSD6,KCD6,other|other|other|other|LEADING_B|other|other|...,1|10|2|3|4|5|6|7|8|9,10,OR,ARID5B,KLF5
1,ARID5B::LEF1,MM,COLO,other|LEADING_B|other|OR|OR|AND|other|other|AN...,1|10|2|3|4|5|6|7|8|9,10,AND,ARID5B,LEF1
2,ARID5B::NFE2L1,MM,COLO,other|other|other|other|other|other|other|othe...,1|10|2|3|4|5|6|7|8|9,10,other,ARID5B,NFE2L1
3,ARID5B::SOX10,MM,COLO,other|other|LEADING_A|AND|other|other|other|OR...,1|10|2|3|4|5|6|7|8|9,10,OR,ARID5B,SOX10
4,ARID5B::STAT1,MM,COLO,other|other|other|other|other|other|other|othe...,1|10|2|3|4|5|6|7|8|9,10,other,ARID5B,STAT1
...,...,...,...,...,...,...,...,...,...
234,TFAP2B::ZNF589,Melanocytes,MC,other|other|other|other|other|other|OR|other|o...,1|10|2|3|4|5|6|7|8|9,10,other,TFAP2B,ZNF589
235,TFAP2C::ZBTB14,SCC,A431,other|other|other|other|other|other|LEADING_B|...,1|10|2|3|4|5|6|7|8|9,10,other,TFAP2C,ZBTB14
236,TFAP2C::ZNF219,SCC,A431,other|other|other|AND|OR|OR|OR|OR,1|10|2|5|6|7|8|9,8,OR,TFAP2C,ZNF219
237,THAP1::THRB,Colon,Colon,other|other|other|OR|AND|other|LEADING_A|other...,1|10|2|3|4|5|6|7|8|9,10,LEADING,THAP1,THRB


In [32]:
def process_vocab(index_string):
    tfs = []
    for tf_config in index_string.split('::'):
        tfs.append(tf_config.split('_')[0])
    return tfs

def find_vocabs(tfs_arr,config='one'):
    if config=='one':
        vocab_df_filt = vocab_df[vocab_df.tf1.isin(tfs_arr)|vocab_df.tf2.isin(tfs_arr)]
    elif config=='both':
        vocab_df_filt = vocab_df[vocab_df.tf1.isin(tfs_arr)&vocab_df.tf2.isin(tfs_arr)]
    else:
        return []
#         raise ValueError('config needs to be one or both')
    return vocab_df_filt.vocab.to_list()

In [63]:
thres_score = 0.5

In [64]:
for tissue, res_arr in tissue_to_res.items():
    print('**********tissue',tissue)
    for res in res_arr:
        print(os.path.basename(res))
        results_df = pd.read_csv(res)
        results_df = results_df[results_df.score>thres_score]
        results_df['tfs'] = results_df['index'].apply(process_vocab)
        results_df['is_mpra_one'] = results_df['tfs'].apply(lambda x: find_vocabs(x))
        results_df['is_mpra_both'] = results_df['tfs'].apply(lambda x: find_vocabs(x,config='both'))
        results_df_filt = results_df[results_df.is_mpra_both.apply(len)>0]
        print(results_df.shape[0], results_df_filt.shape[0])

**********tissue GDSD6
test_GDSD6_purple_boosted_stability_score.csv
12 0
**********tissue Colon
test_Colon_green_boosted_stability_score.csv
14 0
**********tissue Melanocytes
test_Melanocytes_blue_boosted_stability_score.csv
11 0
**********tissue GM12878
test_GM12878_grey_boosted_stability_score.csv
10 0
**********tissue MM
diff_cancer_boosted_stability_score.csv
8 0
**********tissue SCC
diff_cancer_boosted_stability_score.csv
5 0


In [65]:
# after one round new version
tissue_to_res = {
    'GDSD6':[ '../data/processed/fig4_modelling/irf_manual/test_GDSD6_purple_boosted_stability_score.csv'],
    'Colon':['../data/processed/fig4_modelling/irf_manual/test_Colon_green_boosted_stability_score.csv'],
    'Melanocytes':[ '../data/processed/fig4_modelling/irf_manual/test_Melanocytes_blue_boosted_stability_score.csv'],
    'GM12878':['../data/processed/fig4_modelling/irf_manual/test_GM12878_grey_boosted_stability_score.csv'],
    'MM':['../data/processed/fig4_modelling/irf_manual_mm/diff_cancer_boosted_stability_score.csv'],
    'SCC':['../data/processed/fig4_modelling/irf_manual_scc/diff_cancer_boosted_stability_score.csv']
    
}

In [66]:
res_all_df = pd.DataFrame()
for tissue, res_arr in tissue_to_res.items():
    print('**********tissue',tissue)
    for res in res_arr:
        print(os.path.basename(res))
        results_df = pd.read_csv(res)
        results_df = results_df[results_df.score>thres_score]
        results_df['tfs'] = results_df['index'].apply(process_vocab)
        results_df['is_mpra_one'] = results_df['tfs'].apply(lambda x: find_vocabs(x))
        results_df['is_mpra_both'] = results_df['tfs'].apply(lambda x: find_vocabs(x,config='both'))
        results_df_filt = results_df[results_df.is_mpra_one.apply(len)>0]
        results_df_filt['tissue'] = tissue
        print(results_df.shape[0], results_df_filt.shape[0])
        res_all_df = pd.concat([res_all_df,results_df_filt],axis=0)

**********tissue GDSD6
test_GDSD6_purple_boosted_stability_score.csv
12 5
**********tissue Colon
test_Colon_green_boosted_stability_score.csv
14 7
**********tissue Melanocytes
test_Melanocytes_blue_boosted_stability_score.csv
11 1
**********tissue GM12878
test_GM12878_grey_boosted_stability_score.csv
10 4
**********tissue MM
diff_cancer_boosted_stability_score.csv
8 1
**********tissue SCC
diff_cancer_boosted_stability_score.csv
5 2


/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [67]:
res_all_df

,index,score,std,count,tfs,is_mpra_one,is_mpra_both,tissue
562,JUNB_pro,0.697293,NaN,1,[JUNB],"[EGR2::JUNB, JUNB::RORA]",[],GDSD6
596,JUNB_pro::MYNN_pro,0.602208,NaN,1,"[JUNB, MYNN]","[EGR2::JUNB, JUNB::RORA]",[],GDSD6
1420,NFE2L1_pro,0.504808,NaN,1,[NFE2L1],"[ARID5B::NFE2L1, EGR2::NFE2L1, EGR4::NFE2L1, E...",[],GDSD6
1442,NFE2L1_pro::PBX3_pro,0.807692,NaN,1,"[NFE2L1, PBX3]","[ARID5B::NFE2L1, EGR2::NFE2L1, EGR4::NFE2L1, E...",[],GDSD6
1570,NFIA_pro::PBX2_pro,0.744302,NaN,1,"[NFIA, PBX2]",[NFE2L1::NFIA],[],GDSD6
599,JUND_loop,0.585079,NaN,1,[JUND],[JUND::RORA],[],Colon
600,JUN_loop,0.877618,NaN,1,[JUN],"[EGR2::JUN, JUN::RORA, JUN::SREBF2]",[],Colon
639,KLF6_loop,0.751309,NaN,1,[KLF6],[KLF6::SP1],[],Colon
801,MAFG_pro::ZNF770_pro,0.608202,NaN,1,"[MAFG, ZNF770]","[KLF5::MAFG, MAFG::RORA, SP1::ZNF770]",[],Colon
1206,OSR2_pro,0.723822,NaN,1,[OSR2],"[KLF15::OSR2, OSR2::THAP1]",[],Colon


In [69]:
from platform import python_version

print(python_version())

3.7.4
